# Bayes Elo for JTac

In [1]:
using Statistics, LinearAlgebra, Random

In [3]:
import NLopt

In [4]:
sigmoid(d) = 1 / (1 + 10^(d/400))

sigmoid (generic function with 1 method)

In [5]:
struct RankedGame
    player1 :: Int
    player2 :: Int
    result
end

In [6]:
function likelihood(elo1, elo2, result; draw = 0, adv = 0)
    @assert draw >= 0 "Drawing may not carry a negative elo."
    p1 = sigmoid(elo2 - elo1 - adv + draw)
    p2 = sigmoid(elo1 - elo2 + adv + draw)
    if result == 1
        p1
    elseif result == -1
        p2
    else
        1 - p1 - p2
    end
end

likelihood (generic function with 1 method)

In [7]:
function likelihood(eloList, rankedGame :: RankedGame; draw = 0, adv = 0)
    likelihood(eloList[rankedGame.player1], eloList[rankedGame.player2], rankedGame.result, draw = draw, adv = adv)
end

likelihood (generic function with 2 methods)

In [8]:
function log_likelihood(eloList, rankedGames :: Vector{RankedGame}; draw = 0, adv = 0)
    sum( rankedGames ) do rankedGame
        log(likelihood(eloList, rankedGame, draw = draw, adv = adv))
    end
end

log_likelihood (generic function with 1 method)

In [9]:
function log_prior(elo; mean = 0, sd = 1000)
    -0.5 * ((elo - mean) / sd)^2
end

log_prior (generic function with 1 method)

In [10]:
function log_prior(eloList :: Vector; mean = 0, sd = 1000)
    sum(eloList) do elo
        log_prior(elo, mean = mean, sd = sd)
    end
end     

log_prior (generic function with 2 methods)

In [11]:
function objective(eloList, rankedGames; mean = 0, sd = 1000, draw = 0, adv = 0)
    log_likelihood(eloList, rankedGames, draw = draw, adv = adv) +
        log_prior(eloList, mean = mean, sd = sd)
end

objective (generic function with 1 method)

## Simulate some games

In [12]:
using Jtac

In [45]:
k = 10

10

In [46]:
model = RolloutModel()

RolloutModel()

In [47]:
players = [ MCTPlayer(model, power = 10*i) for i in 1:k ]

10-element Array{MCTPlayer{Game},1}:
 MCTPlayer{Game}(RolloutModel(), 10, 1.0f0, "mct10-76781")  
 MCTPlayer{Game}(RolloutModel(), 20, 1.0f0, "mct20-76781")  
 MCTPlayer{Game}(RolloutModel(), 30, 1.0f0, "mct30-76781")  
 MCTPlayer{Game}(RolloutModel(), 40, 1.0f0, "mct40-76781")  
 MCTPlayer{Game}(RolloutModel(), 50, 1.0f0, "mct50-76781")  
 MCTPlayer{Game}(RolloutModel(), 60, 1.0f0, "mct60-76781")  
 MCTPlayer{Game}(RolloutModel(), 70, 1.0f0, "mct70-76781")  
 MCTPlayer{Game}(RolloutModel(), 80, 1.0f0, "mct80-76781")  
 MCTPlayer{Game}(RolloutModel(), 90, 1.0f0, "mct90-76781")  
 MCTPlayer{Game}(RolloutModel(), 100, 1.0f0, "mct100-76781")

In [48]:
game = TicTacToe()

TicTacToe([0, 0, 0, 0, 0, 0, 0, 0, 0], 1, 42)

In [49]:
pvp(players[1], players[2], game)

-1

In [50]:
function createRatedGame(players, game)
    i = rand(1:length(players))
    j = rand(1:length(players)-1)
    if j >= i
        j += 1
    end
    result = pvp(players[i], players[j], game)
    RankedGame(i, j, result)
end

function createRatedGames(players, game, n)
    [createRatedGame(players, game) for _ in 1:n]
end

createRatedGames (generic function with 1 method)

In [51]:
ratings = createRatedGames(players, game, 1000)

1000-element Array{RankedGame,1}:
 RankedGame(4, 9, 1)  
 RankedGame(1, 6, -1) 
 RankedGame(5, 2, 1)  
 RankedGame(4, 6, -1) 
 RankedGame(4, 1, 1)  
 RankedGame(1, 7, -1) 
 RankedGame(2, 10, 1) 
 RankedGame(1, 6, -1) 
 RankedGame(9, 2, 1)  
 RankedGame(4, 10, -1)
 RankedGame(6, 1, 1)  
 RankedGame(2, 8, 1)  
 RankedGame(4, 10, 0) 
 ⋮                    
 RankedGame(3, 7, -1) 
 RankedGame(8, 3, -1) 
 RankedGame(8, 5, 1)  
 RankedGame(10, 2, 1) 
 RankedGame(7, 6, 0)  
 RankedGame(3, 2, 0)  
 RankedGame(8, 5, -1) 
 RankedGame(3, 1, 1)  
 RankedGame(9, 2, 0)  
 RankedGame(8, 6, 1)  
 RankedGame(4, 5, 1)  
 RankedGame(5, 6, -1) 

In [52]:
function wins(p)
    filter( ratings ) do r
        r.player1 == p && r.result == 1 || r.player2 == p && r.result == -1
    end
end

function plays(p)
    filter( ratings ) do r
        r.player1 == p || r.player2 == p
    end
end

winRate(p) = length(wins(p)) / length(plays(p))

winRate (generic function with 1 method)

In [53]:
map(winRate, 1:k)

10-element Array{Float64,1}:
 0.21634615384615385
 0.2736842105263158 
 0.35714285714285715
 0.4020618556701031 
 0.3864734299516908 
 0.41739130434782606
 0.4791666666666667 
 0.4263959390862944 
 0.4264705882352941 
 0.4642857142857143 

## Calculate Elo values

In [98]:
exo = [10*i for i in 1:k]
objective(exo, ratings, draw = 1)

-190.94921476970362

In [100]:
log_prior(exo)

-0.019250000000000003

In [58]:
function estimate_elo(k, ratings; kwargs...)
    opt = NLopt.Opt(:LN_BOBYQA, k)
    NLopt.min_objective!(opt, (x, _) -> -objective(x, ratings; kwargs...))
    NLopt.lower_bounds!(opt, fill(-2000, k))
    NLopt.upper_bounds!(opt, fill(2000, k))
    value, params, ret = NLopt.optimize(opt, [0. for _ in 1:k])
end

function estimate_elo_params(k, ratings; kwargs...)
    opt = NLopt.Opt(:LN_BOBYQA, k+2)
    NLopt.min_objective!(opt, (x, _) -> -objective(x[1:k], ratings; draw = x[k+2], adv = x[k+1], kwargs...))
    NLopt.lower_bounds!(opt, [fill(-2000, k+1); 1])
    NLopt.upper_bounds!(opt, fill(2000, k+2))
    value, params, ret = NLopt.optimize(opt, [5. for _ in 1:k+2])
end

estimate_elo_params (generic function with 1 method)

In [59]:
estimate_elo(k, ratings, draw = 50)

(1053.978314587922, [-168.218, -78.6132, -15.0152, 16.8235, 9.9881, 27.5632, 64.0896, 48.1434, 39.1829, 56.0723], :ROUNDOFF_LIMITED)

In [66]:
estimate_elo_params(k, ratings)

(9519.033927084292, [-135.153, -76.2828, -50.5445, -13.9377, 4.39189, 31.6586, 47.5596, 49.9812, 68.6779, 73.7513, 138.802, 94.4373], :ROUNDOFF_LIMITED)

In [57]:
estimate_elo(k, ratings, draw = 50, adv = 50)

(1017.6700218256246, [-168.35, -80.2983, -16.1494, 17.4908, 7.8439, 33.4092, 60.033, 47.998, 38.1556, 59.901], :ROUNDOFF_LIMITED)

In [67]:
ratings = createRatedGames(players, game, 10000)

10000-element Array{RankedGame,1}:
 RankedGame(9, 7, 1)  
 RankedGame(9, 2, 1)  
 RankedGame(1, 10, -1)
 RankedGame(10, 2, 1) 
 RankedGame(7, 9, 0)  
 RankedGame(8, 7, 0)  
 RankedGame(2, 9, -1) 
 RankedGame(5, 2, 0)  
 RankedGame(1, 4, -1) 
 RankedGame(8, 10, -1)
 RankedGame(6, 7, 1)  
 RankedGame(8, 6, 1)  
 RankedGame(6, 5, 1)  
 ⋮                    
 RankedGame(2, 7, -1) 
 RankedGame(5, 6, 1)  
 RankedGame(7, 6, 0)  
 RankedGame(6, 5, 1)  
 RankedGame(3, 8, 0)  
 RankedGame(3, 2, 1)  
 RankedGame(3, 5, 1)  
 RankedGame(5, 6, 1)  
 RankedGame(5, 4, 1)  
 RankedGame(7, 8, 0)  
 RankedGame(7, 2, 1)  
 RankedGame(2, 4, 1)  